In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import joblib

# Constants
FUEL_DENSITY = 0.991  # kg/L
CO2_FACTOR = 3.114    # g CO2 per g fuel
GT = 14052            # Gross Tonnage of the RO-RO ship

# Load and preprocess data
data = pd.read_csv("sample_data_year.csv", low_memory=False)
data['Time'] = pd.to_datetime(data['Time'], format='%d-%m-%Y %H:%M')
data = data.sort_values('Time').reset_index(drop=True)

# Clean columns
cols = ['FO_ME_Cons', 'Ship_Speed', 'CppPitch', 'Wind_Speed', 'HEEL', 'Fore_Draft', 'Aft_Draft']
for col in cols:
    data[col] = pd.to_numeric(data[col], errors='coerce').ffill()

data = data.dropna()
data = data[data['Ship_Speed'] > 0.5]

# Add derived features
data['Trim'] = data['Aft_Draft'] - data['Fore_Draft']
data['Avg_Draft'] = (data['Fore_Draft'] + data['Aft_Draft']) / 2
data['Fuel_Liters'] = data['FO_ME_Cons'].diff().clip(lower=0)

# Load model and scaler
rf_model = joblib.load('rf_final_model.joblib')
scaler = joblib.load('scaler.joblib')

features = ['CO2_Emission_g', 'Distance_NM', 'Avg_Speed', 'Avg_Wind_Speed',
            'Avg_CppPitch', 'Avg_Heel', 'Avg_Trim', 'Avg_Draft']

# Suggestion function
def generate_suggestion(row):
    suggestions = []
    if row['Trim'] > 1.0:
        suggestions.append("Reduce trim to improve fuel efficiency")
    if row['HEEL'] > 0.5:
        suggestions.append("Balance ballast to reduce heel")
    if row['Wind_Speed'] > 12:
        suggestions.append("Avoid sailing during high wind")
    if row['CppPitch'] < 15:
        suggestions.append("Increase CPP pitch for propulsion efficiency")
    if row['Ship_Speed'] < 17:
        suggestions.append("Maintain optimal cruising speed")
    return "; ".join(suggestions) if suggestions else "Performance is within expected range"

# 1-minute CII Calculation (using GT-based IMO formula)
def calculate_cii_1min(fuel_liters, ship_speed):
    if ship_speed > 0:
        co2 = fuel_liters * FUEL_DENSITY * 1000 * CO2_FACTOR
        distance = ship_speed / 60  # NM per minute
        return co2 / (GT * distance)
    else:
        return np.nan

# Initialize output records
output_log = []

# Run simulation
print("\n --- LIVE CII SIMULATION STARTED ---\n")
cumulative_rows = []

for i in range(1, len(data)):
    row = data.iloc[i]
    fuel = row['Fuel_Liters']
    speed = row['Ship_Speed']

    # Instant CII
    instant_cii = calculate_cii_1min(fuel, speed)

    # Build current cumulative row
    processed_row = {
        'CO2_Emission_g': fuel * FUEL_DENSITY * 1000 * CO2_FACTOR,
        'Distance_NM': speed / 60,
        'Avg_Speed': row['Ship_Speed'],
        'Avg_Wind_Speed': row['Wind_Speed'],
        'Avg_CppPitch': row['CppPitch'],
        'Avg_Heel': row['HEEL'],
        'Avg_Trim': row['Trim'],
        'Avg_Draft': row['Avg_Draft']
    }

    cumulative_rows.append(processed_row)
    cumulative_df = pd.DataFrame(cumulative_rows)
    cum_avg = cumulative_df.mean().to_frame().T
    cum_scaled = scaler.transform(cum_avg)
    predicted_cii = rf_model.predict(cum_scaled)[0]

    suggestions = generate_suggestion(row)

    # Print live to terminal
    print(f" Minute {i}:")
    print(f"    Instant CII     = {instant_cii:.4f}" if not np.isnan(instant_cii) else "    Instant CII     = N/A (zero speed)")
    print(f"    Predicted CII   = {predicted_cii:.4f}")
    print(f"    Suggestions     = {suggestions}")
    print("-" * 50)

    # Append to log
    output_log.append({
        'Minute': i,
        'Time': row['Time'],
        'Instant_CII': instant_cii if not np.isnan(instant_cii) else None,
        'Predicted_CII': predicted_cii,
        'Suggestions': suggestions
    })

    # You can add a timer if running in real-time
    # time.sleep(1)

    # Stop after 60 minutes (or adjust limit)
    if i >= 60:
        break

# Save to CSV
log_df = pd.DataFrame(output_log)
log_df.to_csv("cii_simulation_output.csv", index=False)
print("\n --- LIVE CII SIMULATION COMPLETE ---")
print("Output saved to: cii_simulation_output.csv")



 --- LIVE CII SIMULATION STARTED ---

 Minute 1:
    Instant CII     = 19.1521
    Predicted CII   = 48.6993
    Suggestions     = Performance is within expected range
--------------------------------------------------
 Minute 2:
    Instant CII     = 18.3860
    Predicted CII   = 48.9333
    Suggestions     = Performance is within expected range
--------------------------------------------------
 Minute 3:
    Instant CII     = 17.6200
    Predicted CII   = 48.7022
    Suggestions     = Balance ballast to reduce heel
--------------------------------------------------
 Minute 4:
    Instant CII     = 17.6200
    Predicted CII   = 48.7022
    Suggestions     = Performance is within expected range
--------------------------------------------------
 Minute 5:
    Instant CII     = 16.0878
    Predicted CII   = 48.7004
    Suggestions     = Performance is within expected range
--------------------------------------------------
 Minute 6:
    Instant CII     = 17.6200
    Predicted CII   =